In [1]:
%load_ext autoreload
%autoreload 2

import os
import six

import sys
import pandas as pd
import typing

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 500)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from secure_ai_sandbox_python_lib.session import Session

sandbox_session = Session(session_folder="/tmp/temp_folder", retail_region="NA")
print(sandbox_session.available_clients())
print(sandbox_session.available_resources())
print(sandbox_session.my_owned_s3_bucket_name())
print(sandbox_session.team_owned_s3_bucket_name())

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/04/25 02:19:15] INFO     Found credentials from IAM Role:                                   ]8;id=785930;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=127394;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     CA certs are provided via the AmazonCACerts installation at     ]8;id=988771;file:///home/ec2-user/.local/lib/python3.10/site-packages/amazoncerts/cacerts_helpers.py\cacerts_helpers.py]8;;\:]8;id=31436;file:///home/ec2-user/.local/lib/python3.10/site-packages/amazoncerts/cacerts_helpers.py#109\109]8;;\
                             /home/ec2-user/.local/lib/python3.10/site-packages/amazoncerts                        

Unable to load rust_cloudauth_core Module


                    INFO     Found credentials from IAM Role:                                   ]8;id=540646;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=25848;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[03/04/25 02:19:16] INFO     successfully patched module botocore                                    ]8;id=842306;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/patcher.py\patcher.py]8;;\:]8;id=743394;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/patcher.py#119\119]8;;\

dict_keys(['TRMSModelManagementServiceClient', 'DataAnalyticsWorkflowServiceClient', 'ModelInferenceManagementServiceClient', 'MODSModelWorkflowManagementServiceClient', 'SandboxProxyServiceClient', 'CloudWatch'])
dict_keys(['MDSDataLoader', 'CradleDataLoader', 'MyOwnS3BucketDataLoader', 'SharedBucketS3DataLoader', 'EdxDataLoader', 'FeatureHubDataLoader', 'Docker', 'OTFSimulationTool', 'DAWSWorkflowFilesDownloader', 'MIMSModelRegistrar', 'TagFileUploader', 'AlexandriaDataLoader', 'DataUploader', 'MODSWorkflowBuilder'])
sandboxuserdependency-shemingx-personals3bucket-ti3n3mfs8arg
sandboxdependency-abuse-secureaisandboxteamshare-1l77v9am252um


### evaluation

In [76]:
from sklearn.metrics import roc_auc_score

In [99]:
def calculate_roc_auc(scores, labels, amounts):
    assert len(scores) == len(labels), "Input lengths don't match!"
    return roc_auc_score(labels, scores)


def calculate_partial_auc(scores, labels, amounts, fpr=0.01):
    assert len(scores) == len(labels), "Input lengths don't match!"
    # threshold = model score at 1% FPR
    return roc_auc_score(labels, scores, max_fpr=fpr)


def calculate_dollar_recall(scores, labels, amounts, fpr=0.01):
    assert len(scores) == len(labels) == len(amounts), "Input lengths don't match!"
    # threshold = model score at 1% FPR
    threshold = np.quantile(scores[labels == 0], 1 - fpr)
    abuse_amount_total = amounts[labels == 1].sum()
    abuse_amount_above_threshold = amounts[(labels == 1) & (scores > threshold)].sum()

    dollar_recall = abuse_amount_above_threshold / abuse_amount_total

    return dollar_recall


def calculate_count_recall(scores, labels, amounts, fpr=0.01):
    assert len(scores) == len(labels), "Input lengths don't match!"
    # threshold = model score at 1% FPR
    threshold = np.quantile(scores[labels == 0], 1 - fpr)

    abuse_order_total = len(labels[labels == 1])
    abuse_order_above_threshold = len(labels[(labels == 1) & (scores > threshold)])

    order_count_recall = abuse_order_above_threshold / abuse_order_total

    return order_count_recall

In [140]:
scores_mtl = pd.read_csv("df_test_pred.csv")
scores_mtl.shape

(26817, 14)

scores=pd.read_csv("df_eval_offline.csv",index_col=0)
scores.shape

In [141]:
scores_xgb = pd.read_csv("df_test_xgb_pred.csv")
scores_xgb.shape

(26817, 8)

In [142]:
scores_mtl.head()

,is_abuse,condition,e90,ce90,conc,si90,csi90,is_abuse_pred,condition_pred,e90_pred,ce90_pred,conc_pred,csi90_pred,orderTotalAmountUSD
0,0,0,0,0,0,0.0,0.0,0.070718,0.037485,0.001247,0.000745,0.011287,0.000946,71.66
1,0,0,0,0,0,0.0,0.0,0.116172,0.064038,0.002734,0.001098,0.012910,0.000863,12.91
2,0,0,0,0,0,0.0,0.0,0.108987,0.039475,0.002930,0.001516,0.019764,0.003032,84.02
3,0,0,0,0,0,0.0,0.0,0.740141,0.521142,0.014435,0.001421,0.064510,0.000714,28.38
4,0,0,0,0,0,0.0,0.0,0.116383,0.049084,0.001741,0.000569,0.015178,0.001061,22.69


In [143]:
scores_xgb.head()

,Abuse_type,neg_xgb_pred,condition_xgb_pred,si90_xgb_pred,conc_xgb_pred,e90_xgb_pred,ce90_xgb_pred,csi90_xgb_pred
0,0,0.936985,0.029975,0.020700,0.011116,0.000942,0.000168,0.000114
1,0,0.882189,0.066671,0.041542,0.008964,0.000283,0.000059,0.000291
2,0,0.908577,0.019821,0.028481,0.039636,0.000772,0.002201,0.000512
3,0,0.301171,0.414182,0.226427,0.051893,0.005753,0.000381,0.000192
4,0,0.899858,0.045766,0.045637,0.008205,0.000440,0.000064,0.000029


In [144]:
df = pd.concat([scores_mtl, scores_xgb], axis=1)
df.shape

(26817, 22)

In [145]:
df = df.fillna(0)

In [146]:
rows_with_na = df[df.isna().any(axis=1)]

In [147]:
df["is_abuse_xgb_pred"] = 1 - df.neg_xgb_pred
df.head()

,is_abuse,condition,e90,ce90,conc,si90,csi90,is_abuse_pred,condition_pred,e90_pred,ce90_pred,conc_pred,csi90_pred,orderTotalAmountUSD,Abuse_type,neg_xgb_pred,condition_xgb_pred,si90_xgb_pred,conc_xgb_pred,e90_xgb_pred,ce90_xgb_pred,csi90_xgb_pred,is_abuse_xgb_pred
0,0,0,0,0,0,0.0,0.0,0.070718,0.037485,0.001247,0.000745,0.011287,0.000946,71.66,0,0.936985,0.029975,0.020700,0.011116,0.000942,0.000168,0.000114,0.063015
1,0,0,0,0,0,0.0,0.0,0.116172,0.064038,0.002734,0.001098,0.012910,0.000863,12.91,0,0.882189,0.066671,0.041542,0.008964,0.000283,0.000059,0.000291,0.117811
2,0,0,0,0,0,0.0,0.0,0.108987,0.039475,0.002930,0.001516,0.019764,0.003032,84.02,0,0.908577,0.019821,0.028481,0.039636,0.000772,0.002201,0.000512,0.091423
3,0,0,0,0,0,0.0,0.0,0.740141,0.521142,0.014435,0.001421,0.064510,0.000714,28.38,0,0.301171,0.414182,0.226427,0.051893,0.005753,0.000381,0.000192,0.698829
4,0,0,0,0,0,0.0,0.0,0.116383,0.049084,0.001741,0.000569,0.015178,0.001061,22.69,0,0.899858,0.045766,0.045637,0.008205,0.000440,0.000064,0.000029,0.100142


In [148]:
df.csi90.value_counts()

csi90
0.0    26778
1.0       39
Name: count, dtype: int64

In [149]:
tag_lst = ["is_abuse", "condition", "e90", "ce90", "conc", "csi90"]
amounts_tag = "orderTotalAmountUSD"

In [150]:
for tag in tag_lst:
    tag_pred = f"{tag}_pred"
    tag_pred_xgb = f"{tag}_xgb_pred"

    roc_auc = calculate_roc_auc(df[tag_pred], df[tag], df[amounts_tag])
    partial_auc = calculate_partial_auc(df[tag_pred], df[tag], df[amounts_tag])
    count_recall = calculate_count_recall(df[tag_pred], df[tag], df[amounts_tag])
    dollar_recall = calculate_dollar_recall(df[tag_pred], df[tag], df[amounts_tag])

    roc_auc_xgb = calculate_roc_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    partial_auc_xgb = calculate_partial_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    count_recall_xgb = calculate_count_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )
    dollar_recall_xgb = calculate_dollar_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )

    data = [count_recall, dollar_recall, roc_auc, partial_auc]
    data_xgb = [count_recall_xgb, dollar_recall_xgb, roc_auc_xgb, partial_auc_xgb]

    index = ["count_recall", "dollar_recall", "roc_auc", "partial_auc"]
    mtgbm_eval_result = pd.Series(data, index=index)
    XGB_eval_result = pd.Series(data_xgb, index=index)

    eval_results = pd.DataFrame()
    eval_results["mtgbm_eval_result"] = mtgbm_eval_result
    eval_results["XGB_eval_result"] = XGB_eval_result
    eval_results.to_csv(f"{tag} comparison.csv")
    print("--------------------------")
    print(tag)
    print(eval_results)

--------------------------
is_abuse
               mtgbm_eval_result  XGB_eval_result
count_recall            0.083821         0.083604
dollar_recall           0.374892         0.371988
roc_auc                 0.716268         0.716326
partial_auc             0.522716         0.524035
--------------------------
condition
               mtgbm_eval_result  XGB_eval_result
count_recall            0.109139         0.110272
dollar_recall           0.488480         0.474406
roc_auc                 0.766276         0.765999
partial_auc             0.530921         0.530133
--------------------------
e90
               mtgbm_eval_result  XGB_eval_result
count_recall            0.330645         0.354839
dollar_recall           0.437190         0.457997
roc_auc                 0.849747         0.851419
partial_auc             0.618763         0.617970
--------------------------
ce90
               mtgbm_eval_result  XGB_eval_result
count_recall            0.511111         0.466667
dollar_recall 

In [139]:
for tag in tag_lst:
    tag_pred = f"{tag}_pred"
    tag_pred_xgb = f"{tag}_xgb_pred"

    roc_auc = calculate_roc_auc(df[tag_pred], df[tag], df[amounts_tag])
    partial_auc = calculate_partial_auc(df[tag_pred], df[tag], df[amounts_tag])
    count_recall = calculate_count_recall(df[tag_pred], df[tag], df[amounts_tag])
    dollar_recall = calculate_dollar_recall(df[tag_pred], df[tag], df[amounts_tag])

    roc_auc_xgb = calculate_roc_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    partial_auc_xgb = calculate_partial_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    count_recall_xgb = calculate_count_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )
    dollar_recall_xgb = calculate_dollar_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )

    data = [count_recall, dollar_recall, roc_auc, partial_auc]
    data_xgb = [count_recall_xgb, dollar_recall_xgb, roc_auc_xgb, partial_auc_xgb]

    index = ["count_recall", "dollar_recall", "roc_auc", "partial_auc"]
    mtgbm_eval_result = pd.Series(data, index=index)
    XGB_eval_result = pd.Series(data_xgb, index=index)

    eval_results = pd.DataFrame()
    eval_results["mtgbm_eval_result"] = mtgbm_eval_result
    eval_results["XGB_eval_result"] = XGB_eval_result
    eval_results.to_csv(f"{tag} comparison.csv")
    print("--------------------------")
    print(tag)
    print(eval_results)

--------------------------
is_abuse
               mtgbm_eval_result  XGB_eval_result
count_recall            0.080355         0.083604
dollar_recall           0.372923         0.371988
roc_auc                 0.715355         0.716326
partial_auc             0.522071         0.524035
--------------------------
condition
               mtgbm_eval_result  XGB_eval_result
count_recall            0.109517         0.110272
dollar_recall           0.486458         0.474406
roc_auc                 0.766655         0.765999
partial_auc             0.531143         0.530133
--------------------------
e90
               mtgbm_eval_result  XGB_eval_result
count_recall            0.346774         0.354839
dollar_recall           0.453002         0.457997
roc_auc                 0.853664         0.851419
partial_auc             0.616833         0.617970
--------------------------
ce90
               mtgbm_eval_result  XGB_eval_result
count_recall            0.533333         0.466667
dollar_recall 

In [109]:
for tag in tag_lst:
    tag_pred = f"{tag}_pred"
    tag_pred_xgb = f"{tag}_xgb_pred"

    roc_auc = calculate_roc_auc(df[tag_pred], df[tag], df[amounts_tag])
    partial_auc = calculate_partial_auc(df[tag_pred], df[tag], df[amounts_tag])
    count_recall = calculate_count_recall(df[tag_pred], df[tag], df[amounts_tag])
    dollar_recall = calculate_dollar_recall(df[tag_pred], df[tag], df[amounts_tag])

    roc_auc_xgb = calculate_roc_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    partial_auc_xgb = calculate_partial_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    count_recall_xgb = calculate_count_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )
    dollar_recall_xgb = calculate_dollar_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )

    data = [count_recall, dollar_recall, roc_auc, partial_auc]
    data_xgb = [count_recall_xgb, dollar_recall_xgb, roc_auc_xgb, partial_auc_xgb]

    index = ["count_recall", "dollar_recall", "roc_auc", "partial_auc"]
    mtgbm_eval_result = pd.Series(data, index=index)
    XGB_eval_result = pd.Series(data_xgb, index=index)

    eval_results = pd.DataFrame()
    eval_results["mtgbm_eval_result"] = mtgbm_eval_result
    eval_results["XGB_eval_result"] = XGB_eval_result
    eval_results.to_csv(f"{tag} comparison.csv")
    print("--------------------------")
    print(tag)
    print(eval_results)

--------------------------
is_abuse
               mtgbm_eval_result  XGB_eval_result
count_recall            0.086636         0.083604
dollar_recall           0.402417         0.371988
roc_auc                 0.713942         0.716326
partial_auc             0.523083         0.524035
--------------------------
condition
               mtgbm_eval_result  XGB_eval_result
count_recall            0.115181         0.110272
dollar_recall           0.486972         0.474406
roc_auc                 0.764819         0.765999
partial_auc             0.530853         0.530133
--------------------------
e90
               mtgbm_eval_result  XGB_eval_result
count_recall            0.282258         0.354839
dollar_recall           0.307871         0.457997
roc_auc                 0.787134         0.851419
partial_auc             0.606396         0.617970
--------------------------
ce90
               mtgbm_eval_result  XGB_eval_result
count_recall            0.444444         0.466667
dollar_recall 

In [ ]:
for tag in tag_lst:
    tag_pred = f"{tag}_pred"
    tag_pred_xgb = f"{tag}_xgb_pred"

    roc_auc = calculate_roc_auc(df[tag_pred], df[tag], df[amounts_tag])
    partial_auc = calculate_partial_auc(df[tag_pred], df[tag], df[amounts_tag])
    count_recall = calculate_count_recall(df[tag_pred], df[tag], df[amounts_tag])
    dollar_recall = calculate_dollar_recall(df[tag_pred], df[tag], df[amounts_tag])

    roc_auc_xgb = calculate_roc_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    partial_auc_xgb = calculate_partial_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    count_recall_xgb = calculate_count_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )
    dollar_recall_xgb = calculate_dollar_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )

    data = [count_recall, dollar_recall, roc_auc, partial_auc]
    data_xgb = [count_recall_xgb, dollar_recall_xgb, roc_auc_xgb, partial_auc_xgb]

    index = ["count_recall", "dollar_recall", "roc_auc", "partial_auc"]
    mtgbm_eval_result = pd.Series(data, index=index)
    XGB_eval_result = pd.Series(data_xgb, index=index)

    eval_results = pd.DataFrame()
    eval_results["mtgbm_eval_result"] = mtgbm_eval_result
    eval_results["XGB_eval_result"] = XGB_eval_result
    eval_results.to_csv(f"{tag} comparison.csv")
    print("--------------------------")
    print(tag)
    print(eval_results)

In [148]:
for tag in tag_lst:
    tag_pred = f"{tag}_pred"
    tag_pred_xgb = f"{tag}_xgb_pred"

    roc_auc_mtgbm = calculate_roc_auc(df[tag_pred], df[tag], df[amounts_tag])
    partial_auc_mtgbm = calculate_partial_auc(df[tag_pred], df[tag], df[amounts_tag])
    count_recall_mtgbm = calculate_count_recall(df[tag_pred], df[tag], df[amounts_tag])
    dollar_recall_mtgbm = calculate_dollar_recall(
        df[tag_pred], df[tag], df[amounts_tag]
    )

    roc_auc_mtgbm_bears = calculate_roc_auc(
        df[tag_pred_bears], df[tag], df[amounts_tag]
    )
    partial_auc_mtgbm_bears = calculate_partial_auc(
        df[tag_pred_bears], df[tag], df[amounts_tag]
    )
    count_recall_mtgbm_bears = calculate_count_recall(
        df[tag_pred_bears], df[tag], df[amounts_tag]
    )
    dollar_recall_mtgbm_bears = calculate_dollar_recall(
        df[tag_pred_bears], df[tag], df[amounts_tag]
    )

    roc_auc_xgb = calculate_roc_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    partial_auc_xgb = calculate_partial_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    count_recall_xgb = calculate_count_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )
    dollar_recall_xgb = calculate_dollar_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )

    data = [count_recall, dollar_recall, roc_auc, partial_auc]
    data_xgb = [count_recall_xgb, dollar_recall_xgb, roc_auc_xgb, partial_auc_xgb]

    index = ["count_recall", "dollar_recall", "roc_auc", "partial_auc"]
    online_eval_result = pd.Series(data, index=index)
    XGB_eval_result = pd.Series(data_xgb, index=index)

    eval_results = pd.DataFrame()
    eval_results["mtgbm_eval_result"] = online_eval_result
    eval_results["XGB_eval_result"] = XGB_eval_result
    eval_results.to_csv(f"{tag} comparison.csv")
    print("--------------------------")
    print(tag)
    print(eval_results)

--------------------------
is_abusive_dnr
               online_eval_result  XGB_eval_result
count_recall             0.303654         0.325127
dollar_recall            0.171325         0.244913
roc_auc                  0.942681         0.947632
partial_auc              0.597579         0.607655
--------------------------
is_abusive_pda
               online_eval_result  XGB_eval_result
count_recall             0.238173         0.192404
dollar_recall            0.374988         0.242747
roc_auc                  0.895845         0.744621
partial_auc              0.576594         0.561994
--------------------------
is_abusive_rr
               online_eval_result  XGB_eval_result
count_recall             0.166112         0.200816
dollar_recall            0.221768         0.194130
roc_auc                  0.936953         0.927976
partial_auc              0.546560         0.560853
--------------------------
is_abusive_flr
               online_eval_result  XGB_eval_result
count_recall     

In [128]:
for tag in tag_lst:
    tag_pred = f"{tag}_pred"
    tag_pred_xgb = f"{tag}_xgb_pred"

    roc_auc = calculate_roc_auc(df[tag_pred], df[tag], df[amounts_tag])
    partial_auc = calculate_partial_auc(df[tag_pred], df[tag], df[amounts_tag])
    count_recall = calculate_count_recall(df[tag_pred], df[tag], df[amounts_tag])
    dollar_recall = calculate_dollar_recall(df[tag_pred], df[tag], df[amounts_tag])

    roc_auc_xgb = calculate_roc_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    partial_auc_xgb = calculate_partial_auc(df[tag_pred_xgb], df[tag], df[amounts_tag])
    count_recall_xgb = calculate_count_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )
    dollar_recall_xgb = calculate_dollar_recall(
        df[tag_pred_xgb], df[tag], df[amounts_tag]
    )

    data = [count_recall, dollar_recall, roc_auc, partial_auc]
    data_xgb = [count_recall_xgb, dollar_recall_xgb, roc_auc_xgb, partial_auc_xgb]

    index = ["count_recall", "dollar_recall", "roc_auc", "partial_auc"]
    online_eval_result = pd.Series(data, index=index)
    XGB_eval_result = pd.Series(data_xgb, index=index)

    eval_results = pd.DataFrame()
    eval_results["online_eval_result"] = online_eval_result
    eval_results["XGB_eval_result"] = XGB_eval_result
    eval_results.to_csv(f"{tag} comparison.csv")
    print("--------------------------")
    print(tag)
    print(eval_results)

--------------------------
is_abusive_dnr
               online_eval_result  XGB_eval_result
count_recall             0.312224         0.325127
dollar_recall            0.185605         0.244913
roc_auc                  0.943666         0.947632
partial_auc              0.600583         0.607655
--------------------------
is_abusive_pda
               online_eval_result  XGB_eval_result
count_recall             0.252308         0.192404
dollar_recall            0.367547         0.242747
roc_auc                  0.898972         0.744621
partial_auc              0.580686         0.561994
--------------------------
is_abusive_rr
               online_eval_result  XGB_eval_result
count_recall             0.170231         0.200816
dollar_recall            0.240327         0.194130
roc_auc                  0.938458         0.927976
partial_auc              0.548349         0.560853
--------------------------
is_abusive_flr
               online_eval_result  XGB_eval_result
count_recall     